I got the following code and information from https://www.sdss.org/dr16/irspec/spectro_data/#SummaryCatalogs

# Finding Red Giants

Imports and Variable/Strings

In [2]:
import numpy
from astropy.io import fits 
star_hdus = fits.open('../../data/allStar-r12-l33.fits')
star = star_hdus[1].data
star_hdus.close()
badbits = 2**23
gd = (numpy.bitwise_and(star['aspcapflag'], badbits) == 0) & (star['extratarg']==0)
ind = numpy.where(gd)[0] 

In [5]:
#!/usr/bin/env python
# coding: utf-8

import fitsio
from matplotlib import pyplot as plt
import numpy as np
data = fitsio.read("../../data/allStar-r12-l33.fits")

starbad = 2**23 #bit flag for bad stars 
gd = np.bitwise_and(data["ASPCAPFLAG"], starbad) == 0 
teff_logg_check = np.logical_and(data["TEFF"] > 0, data["LOGG"] > -10) # this checks for -9999 values
teff_logg_feh_check = np.logical_and(data["FE_H"]> -6, teff_logg_check)

indices = np.where(np.logical_and(gd, teff_logg_feh_check)) 
good = data[indices] # this only the good data now
Teff_vals = good["TEFF"]
logg_vals = good["LOGG"]
FeH_vals = good["FE_H"]

ModuleNotFoundError: No module named 'pfits'

In [ ]:
#!/usr/bin/env python
# coding: utf-8

import fitsio
from matplotlib import pyplot as plt
import numpy as np
data = fitsio.read("allStar-r12-l33.fits")

starbad = 2**23 #bit flag for bad stars 
gd = np.bitwise_and(data["ASPCAPFLAG"], starbad) == 0 
teff_logg_check = np.logical_and(data["TEFF"] > 0, data["LOGG"] > -10) # this checks for -9999 values
teff_logg_feh_check = np.logical_and(data["FE_H"]> -6, teff_logg_check)

indices = np.where(np.logical_and(gd, teff_logg_feh_check)) 
good = data[indices] # this only the good data now
Teff_vals = good["TEFF"]
logg_vals = good["LOGG"]
FeH_vals = good["FE_H"]

In [1]:
from astropy import units as u
from astropy.coordinates import SkyCoord

In [2]:
#clusters=[['ngc 2419',12],['Pal 3',12],['Pal 4',12],['ngc 4147',12],['ngc 5024',12],['M53',12],['ngc 5053',12],['ngc 5272',12]
#      ,['M3',12],['ngc 5466',12],['Pal 5',12],['ngc 5904',12],['M5',12],['Pal 14',12],['ngc 6205',12],['M13',12],
#     ['ngc 6341',12],['M92',12],['ngc 6838',12],['M71',12],['ngc 7006',12],['ngc 7078',12],['M15',12],['ngc 7089',12],['M2',12]]
clusters=[['ngc 2419',2.92],['Pal 3',6.18],['Pal 4',2]]

print(len(clusters))
i=2
print(clusters[i])

sclus=clusters[i]
print(sclus[0])
print(sclus[1])

3
['Pal 4', 2]
Pal 4
2


In [3]:

rgs=[['119 Tau',12]]
#clusters=[['ngc 2419',2.92],['Pal 3',6.18],['Pal 4',2]]

print(len(clusters))
i=2
print(clusters[i])

sclus=clusters[i]
print(sclus[0])
print(sclus[1])

3
['Pal 4', 2]
Pal 4
2


In [4]:
radeg= [None] * len(clusters)
decdeg= [None] * len(clusters)
i=0
for clus in clusters:
    #print(clus)
    print(clus[0])
    #print(clus[1])
    clus_center = SkyCoord.from_name(clus[0])
    #print(clus_center.ra, clus_center.dec)
    #print(clus_center.ra.hour, clus_center.dec)
    print(str(clus_center.ra.deg), str(clus_center.dec.deg))
    radeg[i]=clus_center.ra.deg
    decdeg[i]=clus_center.dec.deg
    i=i+1
    clus[1]=10
    
    #im_size = clus[1]*u.arcmin 
    #im_pixels = 1024 
    #cutoutbaseurl = 'http://skyservice.pha.jhu.edu/DR12/ImgCutout/getjpeg.aspx'
    #query_string = urlencode(dict(ra=clus_center.ra.deg, 
    #                          dec=clus_center.dec.deg, 
    #                          width=im_pixels, height=im_pixels, 
    #                          scale=im_size.to(u.arcsec).value/im_pixels))
    #url = cutoutbaseurl + '?' + query_string

    # this downloads the image to your disk
    #urlretrieve(url, 'PAL3_SDSS_cutout.jpg')
    #imfilename='rgout/'+clus[0]+'.jpg'
    #urlretrieve(url, imfilename)

ngc 2419
114.53545833 38.88191667
Pal 3
151.379 0.07094444
Pal 4
172.32 28.97361111


In [5]:
from astroquery.sdss import SDSS
from astropy.io import ascii

q1 = " select s.objid, s.ra, s.dec, s.u, s.g, s.r, s.i, s.z from star s, "
q3 = "  n where s.objid = n.objid "
i=0
for clus in clusters:
    #query = "select top 10                        z, ra, dec, bestObjID                      from                        specObj                      where                        class = 'galaxy'                        and z > 0.3                        and zWarning = 0"
    #query = " select s.objid, s.ra, s.dec, s.u, s.g, s.r, s.i, s.z from star s, dbo.fGetNearbyObjEq(229.0128,-0.1082,3) n where s.objid = n.objid "
    q2 = " dbo.fGetNearbyObjEq("+str(radeg[i])+","+str(decdeg[i])+",8) "
    query = q1+q2+q3
    i=i+1
    res = SDSS.query_sql(query)
    outfile='rgout/'+clus[0]+'.csv'
    ascii.write(res,outfile)

gzip was not found on your system! You should solve this issue for astroquery.eso to be at its best!
On POSIX system: make sure gzip is installed and in your path!On Windows: same for 7-zip (http://www.7-zip.org)!


In [6]:
print(res)


       objid               ra              dec        ...    i        z    
------------------- ---------------- ---------------- ... -------- --------
1237667211061428338 172.248783154785  29.016977490102 ... 20.09388 20.23998
1237667211061429267 172.221763553627 29.0183213925301 ... 22.31523 23.10095
1237667211061428650 172.210561168824 28.9696136908014 ... 22.62903   22.145
1237667211061428660 172.212228454625 28.9698510209076 ... 23.28109 22.04665
1237667211061428712 172.235144561811 28.9671820349401 ... 19.00416 18.49835
1237667211061494736 172.369480184003 29.0093206209201 ... 22.32587 23.75194
1237667211061494090 172.369967527838 29.0236005393859 ... 22.61952 23.56167
1237667211061494106 172.377099409577 29.0230302988427 ... 22.47577 21.66122
1237667211061494136 172.386704992373 29.0307274729142 ... 21.45578 21.82044
1237667211061428989 172.313453285795 29.0264308006407 ... 24.18897 22.98672
                ...              ...              ... ...      ...      ...
123766725347

In [8]:
print (res[1])

       objid               ra              dec           u        g        r        i        z    
------------------- ---------------- ---------------- -------- -------- -------- -------- --------
1237667211061429267 172.221763553627 29.0183213925301 25.29737 25.30601 25.04325 22.31523 23.10095


In [12]:
print(res[1][2])
print(len(res))

29.0183213925301
565


In [25]:
from astropy.table import QTable, Table, Column

clussize=len(res)
startab = Table(names=('objid', 'ra', 'dec', 'u', 'g', 'r', 'i', 'z'),dtype=('int32', 'float64','float64', 'float64','float64','float64', 'float64','float64'))
# t = Table(names=('a', 'b', 'c'), dtype=('f4', 'i4', 'S2'))
for i in range(clussize):
#for i in range(5):
    g=res[i][4]
    r=res[i][5]
    u=res[i][3]
    if g-r>0.8 and g-r<1.3:
        if u-g>0.84*(g-r)+1.758:
            if u-g<2.4*(g-r)+0.73:
                print("about time!")
                startab.add_row(res[i])
        
    

about time!
about time!
about time!
about time!
about time!
about time!
about time!
about time!
about time!
about time!
about time!


In [26]:
print(startab)

  objid           ra              dec        ...    r        i        z    
---------- ---------------- ---------------- ... -------- -------- --------
1635189294 172.252826641857 29.0063031628907 ... 21.16535  20.5558 20.25026
1098122050  172.24111258085 28.8720361473859 ... 22.12561 21.83517 21.60674
1635189585 172.337715748462 28.9920641020658 ... 21.66274 21.53074 20.95118
1635188884 172.315344949426 28.9731051914916 ... 18.84352 18.48847 18.32708
1635188882  172.31167702426 28.9755759486245 ... 18.23309 17.84693 17.55528
1635188879 172.311255584963  28.970681152567 ... 17.35341 16.41033 15.92467
1098186874 172.314679370383 28.9629462891173 ...    17.88 17.39557 17.14681
1635254361 172.434063726624 29.0587151490484 ... 18.84081 18.23163 17.94285
1098186811 172.332501667662 28.8976377003353 ... 17.86664 17.43954 17.18964
1098186962 172.338216399358 28.9429731859855 ...  19.0919 18.20432 17.73378
1098187395 172.366428850052 28.9525824591692 ... 21.97215 21.53922 21.29676


In [28]:
#We will make a loop over all the clusters and find more
#red giants that arent a billion lightyears away.

In [ ]:
#method is in globularcluster_plothr_loop